In [1]:
import pandas as pd
import os
from selenium import webdriver
import time
import re # 정규식을 활용하여 괄호 안 숫자 추출

#os.chdir("")

In [20]:
data = pd.read_csv("상권 데이터")
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + " " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,역,검색어
0,도전최강달인왕만두,한식,한식/백반/한정식,서울특별시 강동구 고덕로 333,5호선 상일동,373.362877,상일동,상일동 도전최강달인왕만두
1,이때,한식,한식/백반/한정식,서울특별시 마포구 동교로38안길 7,경의중앙 홍대입구,360.468770,홍대입구,홍대입구 이때
2,L.A.D,한식,한식/백반/한정식,서울특별시 마포구 잔다리로3안길 23,2호선 홍대입구,583.594176,홍대입구,홍대입구 L.A.D
3,제이씨에스푸드,한식,한식/백반/한정식,서울특별시 관악구 신림로14길 3,2호선 봉천,860.540255,봉천,봉천 제이씨에스푸드
4,비지트,한식,한식/백반/한정식,서울특별시 서초구 동광로18길 82,2호선 방배,810.975643,방배,방배 비지트
...,...,...,...,...,...,...,...,...
135805,얼음과자,패스트푸드,아이스크림판매,서울특별시 강서구 금낭화로 135,5호선 방화,27.430119,방화,방화 얼음과자
135806,오이소서울,패스트푸드,아이스크림판매,서울특별시 강동구 고덕로80길 123,5호선 상일동,983.009054,상일동,상일동 오이소서울
135807,얼음왕국명일14,패스트푸드,아이스크림판매,서울특별시 강동구 상암로 214,5호선 굽은다리(강동구민회관앞),347.273163,굽은다리,굽은다리 얼음왕국명일14
135808,얼음과자,패스트푸드,아이스크림판매,서울특별시 강동구 고덕로83길 6,5호선 상일동,1380.011573,상일동,상일동 얼음과자


In [53]:
data.groupby("검색어").count().sort_values("상호명",ascending=False).head(700)

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,역
검색어,,,,,,,
홍대입구 간단하지만특별한화피디Cafe,18,18,18,18,18,18,18
언주 간단하지만특별한화피디Cafe,15,15,15,15,15,15,15
가좌 간단하지만특별한화피디Cafe,14,14,14,14,14,14,14
어린이대공원 간단하지만특별한화피디Cafe,14,14,14,14,14,14,14
연신내 간단하지만특별한화피디Cafe,14,14,14,14,14,14,14
...,...,...,...,...,...,...,...
독립문 간단하지만특별한화피디Cafe,3,3,3,3,3,3,3
흑석 카페,3,3,3,3,3,3,3
어린이대공원 탐앤탐스,3,3,3,3,3,3,3


In [12]:
driver = webdriver.Chrome("chromedriver")
driver.maximize_window() # 창 크기 최대화

rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
user_review_id = {}
review_json = {}
image_json = {}

items = ["공릉역 공릉순두부","공릉역 찬이네"]
for item in items:
    driver.get("https://www.google.com/maps/") # 구글 지도 접속하기
    searchbox = driver.find_element_by_css_selector("input#searchboxinput") # 검색창에 입력하기
    searchbox.send_keys(item)
    searchbutton = driver.find_element_by_class_name("searchbox-searchbutton") # 검색버튼 누르기
    searchbutton.click()
    time.sleep(2)
    
    
    try:
        reviewbutton = driver.find_element_by_class_name("widget-pane-link")
        res_content = driver.find_element_by_xpath("//button[@jsaction='pane.rating.category']").text
        reviewbutton.click()
    except:
        print("여러 식당 검색")
        time.sleep(1)
        firstbox = driver.find_elements_by_xpath("//a[@class='place-result-container-place-link']")[0].get_attribute("href")
        driver.get(firstbox)
        time.sleep(1)
        reviewbutton = driver.find_element_by_class_name("widget-pane-link")
        res_content = driver.find_element_by_xpath("//button[@jsaction='pane.rating.category']").text
        reviewbutton.click()
    time.sleep(2)

    # 리뷰 전체 뽑기 위해 스크롤 다운
    while True:
        try:
            driver.find_element_by_class_name('section-loading').click()
            time.sleep(2)
        except:
            print(item + " 스크롤 다운 완료")
            break

    spread_review = driver.find_elements_by_xpath("//button[@jsaction='pane.review.expandReview']") 
    for i in range(len(spread_review)): 
        isTrue = spread_review[i].is_displayed() # 보이는 것인지를 확인 
        print("Element is visible? " + str(isTrue)) 
        if isTrue: 
            spread_review[i].click() 
            print(str(i)+"th more button is clicked and wait 2 secs...") 
            time.sleep(2)
        else:
            print("Element is visible? " + str(isTrue)) 


    # 시간 지연
    time.sleep(3)
    user = driver.find_elements_by_xpath("//a[@class='section-review-reviewer-link']")
    user_review = driver.find_elements_by_xpath("//button[@class='section-review-action-menu']")
    date = driver.find_elements_by_xpath("//span[@class='section-review-publish-date']")
    rating = driver.find_elements_by_xpath("//span[@class='section-review-stars']")

    time.sleep(2)
    review = driver.find_elements_by_xpath("//span[@class='section-review-text']")


    # Image

    image = driver.find_elements_by_xpath("//button[@class='section-review-photo']")
    review_json[item] = {}
    user_review_id[item] = {}
    image_json[item] = {}
 
    for i in range(len(date)):
        if review[i].text is not "":
            user_elem = re.findall("\d+",user[i].get_attribute("href"))[0]
            item_elem = item
            rating_elem = re.findall("\d+",rating[i].get_attribute('aria-label'))[0]
            date_elem = date[i].text
            row = {"UserID":user_elem,"ItemID":item_elem,"Rating":rating_elem,"Timestamp":date_elem}
            row = pd.DataFrame(row, index=[i])
            rating_df = rating_df.append(row,ignore_index=True)
            user_review_id[item][user_elem] = user_review[i].get_attribute("data-review-id")
            review_json[item][user_review[i].get_attribute("data-review-id")] = review[i].text
    for i in range(len(image)):
        image_json[item][image[i].get_attribute("data-review-id")] = re.findall(r'\"(.+?)\"',image[i].get_attribute('style'))[0]

# 크롬창 닫기
driver.close()

여러 식당 검색
공릉역 공릉순두부 스크롤 다운 완료
공릉역 찬이네 스크롤 다운 완료


In [8]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,107313277775324604933,공릉역 공릉순두부,4,11달 전
1,118420402434392385647,공릉역 공릉순두부,5,4달 전
2,107285647276587086824,공릉역 공릉순두부,5,2달 전
3,105957577517197085824,공릉역 찬이네,5,3주 전
4,110752710268626215789,공릉역 찬이네,4,1년 전
5,106322870490524936374,공릉역 찬이네,4,6달 전
6,111274556526078121371,공릉역 찬이네,4,7달 전
7,104023620997646895270,공릉역 찬이네,5,10달 전
8,104272023030669453605,공릉역 찬이네,5,10달 전
9,106666072434559420231,공릉역 찬이네,4,1년 전


In [9]:
user_review_id

{'공릉역 공릉순두부': {'107313277775324604933': 'ChZDSUhNMG9nS0VJQ0FnSUNjNV9iak5BEAE',
  '118420402434392385647': 'ChZDSUhNMG9nS0VJQ0FnSUNpdXFuVVJREAE',
  '107285647276587086824': 'ChZDSUhNMG9nS0VJQ0FnSUN5Nk1lTFRREAE'},
 '공릉역 찬이네': {'105957577517197085824': 'ChZDSUhNMG9nS0VJQ0FnSUR5dkk2bGZnEAE',
  '110752710268626215789': 'ChZDSUhNMG9nS0VJQ0FnSUNzMDY3WktREAE',
  '106322870490524936374': 'ChZDSUhNMG9nS0VJQ0FnSUNDbVpha0xBEAE',
  '111274556526078121371': 'ChdDSUhNMG9nS0VJQ0FnSUQ4Zy02eW9BRRAB',
  '104023620997646895270': 'ChZDSUhNMG9nS0VJQ0FnSURjNkphQ1lnEAE',
  '104272023030669453605': 'ChZDSUhNMG9nS0VJQ0FnSURjdUwzOFJ3EAE',
  '106666072434559420231': 'ChZDSUhNMG9nS0VJQ0FnSURZb3BTQmZBEAE',
  '107743443846109228434': 'ChdDSUhNMG9nS0VJQ0FnSURZMi1qWHBBRRAB',
  '115680438140807930167': 'ChZDSUhNMG9nS0VJQ0FnSUNTOWZyWFNREAE',
  '107597318661894767695': 'ChdDSUhNMG9nS0VJQ0FnSUNZNG83QjlnRRAB'}}

In [10]:
review_json

{'공릉역 공릉순두부': {'ChZDSUhNMG9nS0VJQ0FnSUNjNV9iak5BEAE': '우삼겹순두부찌개 완전 맛남!\n제육은 솔직히 그냥그랬고\n순두부찌개 강추!!\n밥도 무한리필이고\n같이나온 기본  찬들도 맛있었당ㅋㅋ',
  'ChZDSUhNMG9nS0VJQ0FnSUNpdXFuVVJREAE': '반찬이 집반찬처럼 맛있게나온다.\n그냥 순두부가 제일 괜찮다.\n맛있고 저렴한 밥집.',
  'ChZDSUhNMG9nS0VJQ0FnSUN5Nk1lTFRREAE': '이정도면 5점을 줘도 괜찮을거 같네요'},
 '공릉역 찬이네': {'ChZDSUhNMG9nS0VJQ0FnSUR5dkk2bGZnEAE': '공릉에선 곱창집 여기만 감. 양도 많고 맛도 최고',
  'ChZDSUhNMG9nS0VJQ0FnSUNzMDY3WktREAE': '대학가 근처라 싸고 양이 진짜 많아요. 남자 돼지 둘이서 이인분도 많았습니다. 양은 우리가 상상하는 그 맛 맞아요. 누린 맛 없이 맛있습니다.',
  'ChZDSUhNMG9nS0VJQ0FnSUNDbVpha0xBEAE': '양 진짜 많음 맛도 맛있음\n개인적으로 야채곱창보단 소금곱창이 더 나은것같다',
  'ChdDSUhNMG9nS0VJQ0FnSUQ4Zy02eW9BRRAB': '저렴한 가격에 혜자 양!! 공릉에서 곱창집 간다면 여기를 추천.',
  'ChZDSUhNMG9nS0VJQ0FnSURjNkphQ1lnEAE': '가격도 저렴하고 양도 진짜 많습니다.. 야채곱창하고 소금막창 먹었는데, 소금막창이 더 맛있었습니다',
  'ChZDSUhNMG9nS0VJQ0FnSURjdUwzOFJ3EAE': '진짜 최고의 맛집... 맛이 뛰어납니다',
  'ChZDSUhNMG9nS0VJQ0FnSURZb3BTQmZBEAE': '공릉동 서울과학기술대학교 앞 큰길에서 좌측으로 조금가서 골목으로 조금 들어가면 있습니다',
  'ChdDSUhNMG9nS0VJQ0FnSURZMi1qWHBBRRAB': '대학가라서 그런지 가성비가 좋은 곳입니다. 저는 야채곱창보다 

In [11]:
image_json

{'공릉역 공릉순두부': {'ChZDSUhNMG9nS0VJQ0FnSUNjNV9iak5BEAE': 'https://lh5.googleusercontent.com/p/AF1QipPU-lkOb6DNo3APm8f68UcoRHbTjaGE1JYcZmrW=w750-h750-p-k-no'},
 '공릉역 찬이네': {'ChZDSUhNMG9nS0VJQ0FnSUNzMDY3WktREAE': 'https://lh5.googleusercontent.com/p/AF1QipM3ffvlEOl1mjN_dPALnawMHpk7buerWH8ONCLh=w750-h750-p-k-no',
  'ChZDSUhNMG9nS0VJQ0FnSURjNkphQ1lnEAE': 'https://lh5.googleusercontent.com/p/AF1QipM0On4K7JlL6NqdBwjD2Zx0dyTNFJ7ZdTQ_h5eS=w750-h750-p-k-no',
  'ChZDSUhNMG9nS0VJQ0FnSURZb3BTQmZBEAE': 'https://lh5.googleusercontent.com/p/AF1QipMBBQ-klGVXz-T9no7hdy1mX0FDFt8P2uau44TY=w750-h750-p-k-no',
  'ChdDSUhNMG9nS0VJQ0FnSUNZNG83QjlnRRAB': 'https://lh5.googleusercontent.com/p/AF1QipMF4VvHo9MqpDK5iVXehdtIq_85UdZvey6g_iAK=w750-h750-p-k-no'}}